In [ ]:
# problem statement: to predict the selling price of car based on feauters based on dataset 
import pandas as pd
import numpy as np
df = pd.read_csv("../input/vehicle-dataset-from-cardekho/car data.csv")
df.head()

In [ ]:
df.shape

In [ ]:
##to find how many catergorical features are there in unique 
##this can be done in a loop function as well
print(df['Seller_Type'].unique())
print(df['Transmission'].unique())
print(df['Owner'].unique())
print(df['Fuel_Type'].unique())

In [ ]:
##to check missing or null values
df.isnull().sum()
#count of null values is zero

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
#how old is the car? we use (present year - selling year) price
# to create a new derived feature, use  

final_dataset=df[['Year', 'Selling_Price', 'Present_Price', 'Kms_Driven',
       'Fuel_Type', 'Seller_Type', 'Transmission', 'Owner']]

In [ ]:
final_dataset.head()

In [ ]:
#creating a new year column
final_dataset['Current_Year']=2020

In [ ]:
final_dataset.head()

In [ ]:
#to calculate the number of years
final_dataset['no_year']=final_dataset['Current_Year'] - final_dataset['Year']

In [ ]:
final_dataset.head()

In [ ]:
#dropping the year column and inplace is true because the operation has to be permanent
final_dataset.drop(['Year'],axis =1,inplace = True)

In [ ]:
final_dataset.head()

In [ ]:
final_dataset.drop(['Current_Year'],axis =1,inplace = True)

In [ ]:
final_dataset.head()

In [ ]:
#get_dummies
#dummy variable trap
# converting categorical features to onehotencoder
# drop_first : this is done so that first column with dummy variable has to be deleted
final_dataset = pd.get_dummies(final_dataset,drop_first=True)


In [ ]:
final_dataset.head()

In [ ]:
#correlation
final_dataset.corr()

In [ ]:
import seaborn as sns
sns.pairplot(final_dataset)

In [ ]:
#plot heat map 

import matplotlib.pyplot as plt
%matplotlib inline

corrmat=final_dataset.corr() 
top_corr_features=corrmat.index 
plt.figure(figsize=(20,20)) 
g=sns.heatmap(final_dataset[top_corr_features].corr(),annot=True,cmap="RdYlGn")

In [ ]:
final_dataset.head()

In [ ]:
#independent and dependant features
x=final_dataset.iloc[:,1:]
y=final_dataset.iloc[:,0]

In [ ]:
x.head

In [ ]:
y.head()

In [ ]:
### feature importance
from sklearn.ensemble import ExtraTreesRegressor
model = ExtraTreesRegressor()
model.fit(x,y)

In [ ]:
print(model.feature_importances_)

In [ ]:
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_,index = x.columns)
feat_importances.nlargest(5).plot(kind='barh')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)
#test size is taken as 20%

In [ ]:
x_train.shape

In [ ]:
x_train

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_random = RandomForestRegressor ()

In [ ]:
###Hyperparameters
n_estimators=[int(X) for X in np.linspace(start = 100, stop = 1200 , num =12)]
print(n_estimators)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
#Hyperparameterized tuning is done using RandomizedSearchCV
#Used to find best parameters  

In [ ]:
#Randomized Search CV

# Number of trees in random forest
n_estimators=[int(X) for X in np.linspace(start = 100, stop = 1200 , num =12)]
# Number of features to consider at every split
max_features = ['auto','sqrt']
# Maximum number of levels in tree
max_depth = [int(X) for X in np.linspace(5,30,num =6)]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split =[2,5,10,15,100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1,2,5,10]

In [ ]:
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

print(random_grid)

In [ ]:
# Initalizing random forest regressor
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()

In [ ]:
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,scoring='neg_mean_squared_error', n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = 1)

In [ ]:
rf_random.fit(x_train,y_train)

In [ ]:
rf_random.best_params_

In [ ]:
rf_random.best_score_

In [ ]:
predictions=rf_random.predict(x_test)

In [ ]:
predictions

In [ ]:
sns.distplot(y_test-predictions)

In [ ]:
plt.scatter(y_test,predictions)

In [ ]:
from sklearn import metrics

In [ ]:
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [ ]:
import pickle
# open a file, where you ant to store the data
file = open('random_forest_regression_model.pkl', 'wb')

# dump information to that file
pickle.dump(rf_random, file)